In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/4_Compartment_Analysis_2021/Expt2.2_Demographics Analysis/Input')
cur_date = '041721'

library(readxl)
library(tidyverse)
library(reshape2)
library(car)

#reading in file
data_df = data.frame(read_excel("CytokineData_032521.xlsx", sheet = 2))

#reading in demographics file
demographics_data_df = data.frame(read_excel("SubjectInfo_032521.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some




In [2]:
data_df = full_join(data_df, demographics_data_df)
head(data_df)

Joining, by = c("SubjectNo", "Group", "SubjectID")



,Original.Identifier,Protein,Conc,Conc_pslog2,Compartment,SubjectNo,Group,SubjectID,Race,Ethnicity,Sex,Age,BMI,Serum_Cotinine
,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,20005,Eotaxin,108.4988037,6.7747713,NELF,1,NS,NS_1,AA,NH,M,43,41.2,NA
2,20005,Eotaxin3,160.5679919,7.3359976,NELF,1,NS,NS_1,AA,NH,M,43,41.2,NA
3,20005,GMCSF,0.5150623,0.5993771,NELF,1,NS,NS_1,AA,NH,M,43,41.2,NA
4,20005,IFNg,5.5197311,2.7048125,NELF,1,NS,NS_1,AA,NH,M,43,41.2,NA
5,20005,IL10,0.8386947,0.8786819,NELF,1,NS,NS_1,AA,NH,M,43,41.2,NA
6,20005,IL12p40,13.1695722,3.8247243,NELF,1,NS,NS_1,AA,NH,M,43,41.2,NA


# Chi Square Analysis/ ANOVA (Table XX)

Comparing demographic variables to see if there are differences across smoking statuses. 

In [3]:
get_tables = function(df, variable){
    #getting variable stratified by smoking status
    variable_grouped_df = df %>% group_by(Group) %>% select(all_of(variable)) 
    #getting overall variable numbers
    overall_variable_df = df %>% select(all_of(variable)) 
    #adding a col that designates this from the overall cohort
    overall_variable_df$Group = rep('O', times = length(overall_variable_df)) #O stands for overall
    overall_variable_df = overall_variable_df[,c(2,1)]
    
    #needed to make a df, so both had row numbers
    variable_table = table(rbind(overall_variable_df, data.frame(variable_grouped_df)))
    return(variable_table)
}
sex_table = get_tables(demographics_data_df, 'Sex')
race_table = get_tables(demographics_data_df, 'Race')
ethnicity_table = get_tables(demographics_data_df, 'Ethnicity')

#this now gives me weird results
sex_table
race_table
ethnicity_table

Adding missing grouping variables: `Group`

Adding missing grouping variables: `Group`

Adding missing grouping variables: `Group`



      Sex
Group   F  M
  CS    6  8
  Ecig  4 10
  NS   14  6
  O    24 24

      Race
Group  AA As  W
  CS    7  0  7
  Ecig  2  2 10
  NS    2  2 16
  O    11  4 33

      Ethnicity
Group   H NH
  CS    0 14
  Ecig  0 14
  NS    3 17
  O     3 45

In [4]:
#fisher exact test
fisher.test(sex_table)
fisher.test(race_table)
fisher.test(ethnicity_table)


	Fisher's Exact Test for Count Data

data:  sex_table
p-value = 0.1123
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data

data:  race_table
p-value = 0.1754
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data

data:  ethnicity_table
p-value = 0.3176
alternative hypothesis: two.sided


In [5]:
#dichotimized race
#making race binary between white and non-white subjects
demographics_data_df = demographics_data_df %>%
    mutate(Race = ifelse(Race != 'W', 'NW', Race))

#dichotimized race
di_race_table = get_tables(demographics_data_df, 'Race')
fisher.test(di_race_table)

Adding missing grouping variables: `Group`




	Fisher's Exact Test for Count Data

data:  di_race_table
p-value = 0.3313
alternative hypothesis: two.sided


In [6]:
#running anova test for continuous values: age, bmi, cotinine

#putting group into a factor in order to make comparisons
demographics_data_df$Group = factor(demographics_data_df$Group, labels = c(unique(demographics_data_df$Group)))

#standardizing data (this makes the data MORE normalized)
mutated_df = demographics_data_df %>%
    mutate(Serum_Cotinine = ifelse(Serum_Cotinine == 'NA', 0, Serum_Cotinine)) %>%
    mutate(Norm_Age = scale(Age), Norm_BMI = scale(BMI), Norm_Cotinine = scale(as.numeric(Serum_Cotinine)))

#contrasts show what compartments are being compared in anova
#these comparisons aren't what we want so I changed them in the function below
contrasts(mutated_df$Group)

,CS,Ecig
NS,0,0
CS,1,0
Ecig,0,1


                        Table of Contrasts
>                   NS    | CS  |  Ecig  |  Sum
>     Contrast 1 |  -2    |  1  |   1    |   0
>     Contrast 2 |  0     | -1  |   1    |   0

> - Contrast 1: compares NS to all other groups
> - Contrast 2: compares CS and Ecig

In [7]:
anova_age = Anova(aov(Norm_Age~Group, data = mutated_df), type = 'III')
anova_age
anova_bmi = Anova(aov(Norm_BMI~Group, data = mutated_df), type = 'III')
anova_bmi
anova_continine = Anova(aov(Norm_Cotinine~Group, data = mutated_df), type = 'III')
anova_continine

,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),4.228173,1,5.024277,0.029971822
Group,9.130316,2,5.424712,0.007751058
Residuals,37.869684,45,NA,NA


,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.1449576,1,0.1413620,0.7086975
Group,0.8554084,2,0.4170953,0.6614794
Residuals,46.1445916,45,NA,NA


,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),13.22787,1,23.98344,1.293985e-05
Group,22.18062,2,20.10783,5.762288e-07
Residuals,24.81938,45,NA,NA
